In [1]:
'''
Arman Arakelyan - 3/20/2019
'''
import os
import random
import numpy as np
from collections import deque
from CarAIClass import CarAI
import pygame as pg
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from shapely.geometry import LineString
'''
IMPORTANT:
Please read the top comments of each cell to know what each cell does. This is extremely important since some of code is designed to create new tracks and I would like to have you 
run the original track first, since it is relatively simple and does not take too long
I couldn't find any way to save my pretrained model so I had to make something that would run fast. Again please read all top comments before running the code.
'''

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


Using TensorFlow backend.


"\nIMPORTANT:\nPlease read the top comments of each cell to know what each cell does. This is extremely important since some of code is designed to create new tracks and I would like to have you \nrun the original track first, since it is relatively simple and does not take too long\nI couldn't find any way to save my pretrained model so I had to make something that would run fast. Again please read all top comments before running the code.\n"

In [2]:
#Run to setup initial variables
os.environ["CUDA_VISIBLE_DEVICES"] = "cuda:0"
clock = pg.time.Clock()
x = 50
y = 50
width = 20
height = 40
vel = 5
run = True
angle = 0
car = pg.image.load("./Sprites/car.png")
car_rect = car.get_rect(center = (200,600))
drawMode = True
collision = True
rewardGate = False
pointList = []
pointList1 = []
gateLocation = []
gateLocationCopy = []
drawModeCount = 0;
gateNum = 0;
totalReward = 0;

In [3]:
'''
Function used to draw a track in game, run this to be able to see what is happening on the screen
'''
def drawTrack(events, window):
    if(drawMode):
        if(drawModeCount == 0):
            start = (0,0)
            mousePos = pg.mouse.get_pos()
            for event in events:
                if event.type == pg.MOUSEBUTTONDOWN:
                    pointList.append(mousePos)
                    #print("buttonDOWN")
            if(len(pointList) != 0):
                start = pointList[-1]
            pg.draw.aaline(window, (0,0,0), start, mousePos, False)
        elif(drawModeCount == 1):
            start = (0,0)
            mousePos = pg.mouse.get_pos()
            for event in events:
                if event.type == pg.MOUSEBUTTONDOWN:
                    pointList1.append(mousePos)
                    #print("buttonDOWN")
            if(len(pointList1) != 0):
                start = pointList1[-1]
            pg.draw.aaline(window, (0,0,0), start, mousePos, False)
    if(len(pointList)>2):
        pg.draw.polygon(window, (100,100,100), pointList)
    if(len(pointList1)>2):
        pg.draw.polygon(window, (0,150,0), pointList1)

In [4]:
'''
I ended up not using this option since it seemed to confused the AI more, but you can choose to draw reward gates that act as extra guidance for the AI, you can still run it
'''
def drawRewardGates(events, window):
    global rewardGate
    global gateNum
    global gateLocationCopy
    if(rewardGate):
        if(gateNum < 8):
            mousePos = pg.mouse.get_pos()
            for event in events:
                if event.type == pg.MOUSEBUTTONDOWN:
                    newPos = [mousePos[0]+200, mousePos[1]]
                    gateLocation.append((mousePos,newPos))
                    gateNum+=1;
                    if(gateNum >= 8):
                        gateLocationCopy = gateLocation.copy()
                        #print(gateLocationCopy)
        else:
            rewardGate = False
    if(len(gateLocation)!=0):
        for gate in gateLocation:
            pg.draw.aaline(window,(255,0,0),gate[0], gate[1], False)

In [5]:
'''
The root of how car sensors, game ending and all else works
Checks for collisions between lines
Run this for sure
'''
def collisionCheck(endpoint, window, carPos):
    distanceInfo = [];
    reward = 0;
    if(collision):
        line1 = []
        line2 = []
        gate = []
        temp = []
        #print(endpoint)
        for j in range(len(endpoint)):
            line1.append(LineString([
                            (car_rect.center[0], car_rect.center[1]),
                            (endpoint[j][0], endpoint[j][1])
                            ]))
        for k in range(len(pointList)-1):
            line2.append(LineString([
                        (pointList[k][0], pointList[k][1]),
                        (pointList[k+1][0], pointList[k+1][1])
                        ]))
        for k in range(len(pointList1)-1):
            line2.append(LineString([
                        (pointList1[k][0], pointList1[k][1]),
                        (pointList1[k+1][0], pointList1[k+1][1])
                        ]))
        for l1 in line1:
            p = 0
            for l2 in line2:
                collisionPoint = l1.intersection(l2)
                if(l1.intersects(l2)):
                    distanceInfo.append(distanceBetweenTwoPoints(collisionPoint.x, collisionPoint.y, carPos[0], carPos[1]))        
                    pg.draw.circle(window, (255,0,0), (int(collisionPoint.x), int(collisionPoint.y)), 6)    
                    p = 1
                    break;
            if(p == 0):
                distanceInfo.append(200)
        #print(distanceInfo)
        for g in gateLocation:
            gate.append(LineString([
                (g[0][0],g[0][1]),
                (g[1][0],g[1][1])
            ]))
        carPosLine = LineString([
            (carPos[0],carPos[1]),
            (carPos[0],carPos[1]-20)
        ])
        cntr = 0;
        for g1 in gate:
            if(carPosLine.intersects(g1)):
                reward = 10;
                del gateLocation[cntr]
                cntr+=1
    return distanceInfo, reward;

In [6]:
'''
Small helper function
Run this
'''
def distanceBetweenTwoPoints(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [7]:
'''
If you want to create a new track please run this piece,
To create a new track run this segment to empty out the current track points.
DO not run if you are only testing the program
'''
drawMode = True
rewardGate = False
pointList = []
gateLocation = []
pointList1 = []
gateLocationCopy = []
drawModeCount = 0;
GateNum = 0;

In [8]:
'''
AI uses these to move the car
Run this
'''
def forward():
    global car_rect
    global angle
    global vel
    car_rect[0] -= np.sin(np.radians(angle))*vel
    car_rect[1] -= np.cos(np.radians(angle))*vel
def left():
    global angle
    angle += 3
def right():
    global angle
    angle -= 3

In [9]:
#Used to draw track, DO NOT USE THIS AS THE MAIN AI< THIS IS PURELY TO DRAW AND QUIT
'''
Only run this if you are drawing a new track.
once you start the game you will be clicking points around the screen marking the outer perimeter of the track (like a polygon)
make sure your starting point is roughly the same as your ending point.
Once you are done drawing the perimeter press X to stop then press Z to draw the inner part of the track
After pressing Z you would draw the same way you did for the outer track except this time it is going to fill the inside green
Recommended to follow the same points as your outer perimeter but with a bit of distance.
Once done press X again you should see sensors activating on the car, simply close the window.
'''
global getLocation
global totalReward

scores = []

for i in range(1):
#===============================================================================
#===============================================================================
#===============================================================================
#=================================RESET=========================================
    #===========================================================================
    #===========================================================================
    #===========================================================================
    x = 50
    y = 50
    width = 20
    height = 40
    vel = 5
    angle = 0
    car = pg.image.load("./Sprites/car.png")
    car_rect = car.get_rect(center = (200,600))
    window = pg.display.set_mode((1500,800))
    pg.init()
    pg.display.set_caption("RaceAI")
    gateLocation = gateLocationCopy.copy()
    #Enter main game loop
    totalReward = 0
    distanceInfo = []
    run = True
    observation = []
    observation.append(distanceInfo)
    score = 0
    lastAction = 0
    while run:
        clock.tick(30)
        window.fill((0,150,0))
        #Handle Input Events
        events = pg.event.get()
        for event in events:
            if event.type == pg.QUIT:
                run = False
        keys = pg.key.get_pressed()
#======================================TAKE ACTION==========================
        action = 3#np.random.randint(0,2, dtype = int)
#===============================================================================
    #===============================================================================
    #===============================================================================
    #=================================ONE STEP======================================
        #===========================================================================
        #===========================================================================
        #===========================================================================
        if action == 1:
            right()
        elif action == 0:
            left()
        #forward();
        #totalReward+=1
        if(keys[pg.K_x]):
            drawMode = False
        if(keys[pg.K_z]):
            drawMode = True
            drawModeCount = 1
        if(keys[pg.K_o]):
            stop = True;
        #Draw the track
        drawTrack(events, window);
        drawRewardGates(events, window);
        #calculate some car stuff
        rotCar = pg.transform.rotate(car, angle)
        car_rect = rotCar.get_rect(center = car_rect.center)
        endpoint = []  
        #car movement and stuff
        window.blit(rotCar, car_rect)
        for i in range(1, 4):
            degree = angle+i*15
            endX = np.cos(np.radians(degree))*(car_rect.center[0] - car_rect.center[0]) - np.sin(np.radians(degree))*(car_rect.center[1]+150 - car_rect.center[1]) + car_rect.center[0]
            endY = np.sin(np.radians(degree))*(car_rect.center[0] - car_rect.center[0]) - np.cos(np.radians(degree))*(car_rect.center[1]+150 - car_rect.center[1]) + car_rect.center[1]
            endpoint.append((endX, endY))
        for h in range(1, 4):
            degree = angle+h*15+300
            endX = np.cos(np.radians(degree))*(car_rect.center[0] - car_rect.center[0]) - np.sin(np.radians(degree))*(car_rect.center[1]+150 - car_rect.center[1]) + car_rect.center[0]
            endY = np.sin(np.radians(degree))*(car_rect.center[0] - car_rect.center[0]) - np.cos(np.radians(degree))*(car_rect.center[1]+150 - car_rect.center[1]) + car_rect.center[1]
            endpoint.append((endX, endY))
        if(drawModeCount == 1 and not drawMode):
            for point in endpoint:
                pg.draw.aaline(window, (0,0,0), car_rect.center, point, False)
        #check for collisions
        distanceInfo_, reward = collisionCheck(endpoint, window, car_rect.center);
        totalReward += reward
        done = False
        if(len(distanceInfo_)!= 0):
            distanceInfo_ = np.array(distanceInfo_)
            done = (distanceInfo_ <= 20).any()
        if(len(distanceInfo) == 0):
            distanceInfo = distanceInfo_
            #print(distanceInfo)
        #===========================================================================
        #===========================================================================
        #===========================================================================
    #====================================ACCESS RESULTS BASED ON STEP===============
    #===============================================================================
    #===============================================================================
    #===============================================================================
        #current state - distanceInfo
        #action taken - action
        #current reward - totalReward
        #next state - distanceInfo_
        
        
        score += totalReward
        if done:
            totalReward = -1000
        if(done):
            run = False
        pg.display.update()
    scores.append(score)
    print("score: ", score)
    pg.quit()

score:  0


In [10]:
#Saving these variables is saving the track you drew.
#Simply copy what they print per variable
'''
Once you drew your track you can print these and take the points necessary, assigning them in the next cell.
'''
print("drwaMode: ", drawMode)
print("rewardgate: ", rewardGate)
print("pointList: ", pointList)
print("getLocation: ", gateLocation)
print("pointList1: ", pointList1)
print("gateLocationCopy: ", gateLocationCopy)
print("drawModeCOunt: ", drawModeCount)
print("GateNum: ", GateNum)

drwaMode:  False
rewardgate:  False
pointList:  [(115, 628), (121, 140), (245, 16), (442, 18), (561, 191), (786, 195), (924, 30), (1180, 32), (1292, 195), (1295, 509), (1069, 743), (257, 758), (117, 629)]
getLocation:  []
pointList1:  [(294, 611), (254, 197), (380, 177), (514, 331), (867, 339), (990, 143), (1118, 154), (1154, 230), (1149, 482), (1014, 640), (351, 650), (293, 610)]
gateLocationCopy:  []
drawModeCOunt:  1
GateNum:  0


In [ ]:
# MAIN AI
'''
This is the MAIN AI, this by default has a track that is relatively simple and does not take too long to run. Please run this at least once before drawing a new track.
'''
drawMode = False
rewardGate = False
pointList = [(115, 628), (121, 140), (245, 16), (442, 18), (561, 191), (786, 195), (924, 30), (1180, 32), (1292, 195), (1295, 509), (1069, 743), (257, 758), (117, 629)]
pointList1 = [(294, 611), (254, 197), (380, 177), (514, 331), (867, 339), (990, 143), (1118, 154), (1154, 230), (1149, 482), (1014, 640), (351, 650), (293, 610)]
drawModeCount = 1
global getLocation
global totalReward
agent = CarAI(6, 2) #6 sensors, 2 possible moves (left or right)
scores = []
epsilons = []
learnSize = 32;
memSize= 1000
for ai in range(500): #run for 500 games usually < 100 is enough, but just in case
#===============================================================================
#===============================================================================
#===============================================================================
#======RESET DO NOT MODIFY THINGS HERE ITS FOR THE GAME LOOP====================
    #===========================================================================
    #===========================================================================
    #===========================================================================
    x = 50
    y = 50
    width = 20
    height = 40
    vel = 5
    angle = 0
    car = pg.image.load("./Sprites/car.png")
    car_rect = car.get_rect(center = (200,600))
    window = pg.display.set_mode((1500,800))
    pg.init()
    pg.display.set_caption("RaceAI")
    gateLocation = gateLocationCopy.copy()
    #Enter main game loop
    totalReward = 0
    distanceInfo = np.array([150,150,150,150,150,150])
    run = True
    score = 0
    lastAction = 0
    distanceInfo = np.reshape(distanceInfo, [1, 6])
    time = 0
    while run:
        time+=1
        clock.tick(30)
        window.fill((0,150,0))
        #Handle Input Events
        events = pg.event.get()
        for event in events:
            if event.type == pg.QUIT:
                run = False
        keys = pg.key.get_pressed()
#======================================TAKE ACTION==========================
        action = agent.getMove(distanceInfo)
#===============================================================================
    #===============================================================================
    #===============================================================================
    #=====ONE STEP: DO NOT MODIFY THIS IT SERVES AS A FETCH FOR INFORMATIONF OR AI==
        #===========================================================================
        #===========================================================================
        #===========================================================================
        if action == 1:
            right()
        elif action == 0:
            left()
        forward();
        totalReward+=1

        if(keys[pg.K_o]):
            stop = True;
        #Draw the track
        drawTrack(events, window);
        drawRewardGates(events, window);
        #calculate some car stuff
        rotCar = pg.transform.rotate(car, angle)
        car_rect = rotCar.get_rect(center = car_rect.center)
        endpoint = []  
        #car movement and stuff
        window.blit(rotCar, car_rect)
        for i in range(1, 4):
            degree = angle+i*20
            endX = np.cos(np.radians(degree))*(car_rect.center[0] - car_rect.center[0]) - np.sin(np.radians(degree))*(car_rect.center[1]+210 - car_rect.center[1]) + car_rect.center[0]
            endY = np.sin(np.radians(degree))*(car_rect.center[0] - car_rect.center[0]) - np.cos(np.radians(degree))*(car_rect.center[1]+210 - car_rect.center[1]) + car_rect.center[1]
            endpoint.append((endX, endY))
        for h in range(1, 4):
            degree = angle+h*20+280
            endX = np.cos(np.radians(degree))*(car_rect.center[0] - car_rect.center[0]) - np.sin(np.radians(degree))*(car_rect.center[1]+210 - car_rect.center[1]) + car_rect.center[0]
            endY = np.sin(np.radians(degree))*(car_rect.center[0] - car_rect.center[0]) - np.cos(np.radians(degree))*(car_rect.center[1]+210 - car_rect.center[1]) + car_rect.center[1]
            endpoint.append((endX, endY))
        if(drawModeCount == 1 and not drawMode):
            for point in endpoint:
                pg.draw.aaline(window, (0,0,0), car_rect.center, point, False)
        #check for collisions
        distanceInfo_, reward = collisionCheck(endpoint, window, car_rect.center);
        totalReward += reward
        done = False
        if(len(distanceInfo_)!= 0):
            distanceInfo_ = np.array(distanceInfo_)
            done = (distanceInfo_ <= 20).any()
        if(len(distanceInfo) == 0):
            distanceInfo = distanceInfo_
            #print(distanceInfo)
        #===========================================================================
        #===========================================================================
        #===========================================================================
    #====================================ACCESS RESULTS BASED ON STEP===============
    #===============================================================================
    #===============================================================================
    #===============================================================================
        #current state - distanceInfo
        #action taken - action
        #current reward - totalReward
        #next state - distanceInfo_
        score += totalReward
        if(score > 10000000):
            run = False
        if done:
            totalReward = -100
        distanceInfo = np.reshape(distanceInfo, [1, 6])
        distanceInfo_ = np.reshape(distanceInfo_, [1, 6])
        agent.store(distanceInfo, action, totalReward, distanceInfo_, done)
        distanceInfo = distanceInfo_.copy()
        if(done):
            print("Finished game: ", ai, " Epsilon is currently at: ", agent.epsilon)
            run = False
        pg.display.update()
    if len(agent.memory)>learnSize:
            agent.learn(learnSize)
    scores.append(score)
    epsilons.append(agent.epsilon)
    print("Score: ", score)
    pg.quit()
    if(score > 10000000):
        break;



Instructions for updating:
Colocations handled automatically by placer.
Finished game:  0  Epsilon is currently at:  1.0
Instructions for updating:
Use tf.cast instead.
Score:  2016
Finished game:  1  Epsilon is currently at:  0.97
Score:  4656
Finished game:  2  Epsilon is currently at:  0.9409
Score:  1326
Finished game:  3  Epsilon is currently at:  0.912673
Score:  1953
Finished game:  4  Epsilon is currently at:  0.8852928099999999
Score:  5460
Finished game:  5  Epsilon is currently at:  0.8587340256999999
Score:  4278
Finished game:  6  Epsilon is currently at:  0.8329720049289999
Score:  595
Finished game:  7  Epsilon is currently at:  0.8079828447811299
Score:  4950
Finished game:  8  Epsilon is currently at:  0.783743359437696
Score:  1711
Finished game:  9  Epsilon is currently at:  0.7602310586545651
Score:  6441
Finished game:  10  Epsilon is currently at:  0.7374241268949281
Score:  1711
Finished game:  11  Epsilon is currently at:  0.7153014030880802
Score:  6786
Finishe

In [10]:
np.savetxt('scores.txt', np.array(scores), fmt='%d')
np.savetxt('epsilon.txt', np.array(epsilons), fmt='%f')
'''
save the points for plotting later
'''

In [ ]:
'''
sample complex track that the AI has hard time beating due to sharp unusual turns. Just experimental code. It did finish the track after 300 attempts, check the video links for more info.
Feel free to use this segment to run your own tracks after training the AI on the main one. This may take longer depending on the difficulty of the track though.
'''
drawMode = False
rewardGate = False
pointList = [(140, 610), (140, 417), (160, 273), (258, 123), (404, 49), (563, 87), (1331, 85), (1466, 633), (1436, 710), (1316, 766), (1125, 758), (1064, 601), (1031, 486), (927, 418), (723, 464), (716, 747), (136, 744), (141, 597)]
pointList1 = [(260, 623), (276, 305), (414, 164), (541, 201), (1224, 181), (1356, 610), (1291, 658), (1184, 633), (1052, 361), (713, 343), (583, 494), (614, 653), (262, 649), (263, 617)]
drawModeCount = 1
global getLocation
global totalReward
#agent = CarAI(6, 2) #6 sensors, 2 possible moves (left or right)
scores = []
epsilons = []
learnSize = 32;
memSize= 1000
for ai in range(500): #run for 500 games usually < 100 is enough, but just in case
#===============================================================================
#===============================================================================
#===============================================================================
#======RESET DO NOT MODIFY THINGS HERE ITS FOR THE GAME LOOP====================
    #===========================================================================
    #===========================================================================
    #===========================================================================
    x = 50
    y = 50
    width = 20
    height = 40
    vel = 5
    angle = 0
    car = pg.image.load("./Sprites/car.png")
    car_rect = car.get_rect(center = (200,600))
    window = pg.display.set_mode((1500,800))
    pg.init()
    pg.display.set_caption("RaceAI")
    gateLocation = gateLocationCopy.copy()
    #Enter main game loop
    totalReward = 0
    distanceInfo = np.array([150,150,150,150,150,150])
    run = True
    score = 0
    lastAction = 0
    distanceInfo = np.reshape(distanceInfo, [1, 6])
    time = 0
    while run:
        time+=1
        clock.tick(30)
        window.fill((0,150,0))
        #Handle Input Events
        events = pg.event.get()
        for event in events:
            if event.type == pg.QUIT:
                run = False
        keys = pg.key.get_pressed()
#======================================TAKE RANDOM ACTION==========================
        action = agent.getMove(distanceInfo)
#===============================================================================
    #===============================================================================
    #===============================================================================
    #=====ONE STEP: DO NOT MODIFY THIS IT SERVES AS A FETCH FOR INFORMATIONF OR AI==
        #===========================================================================
        #===========================================================================
        #===========================================================================
        if action == 1:
            right()
        elif action == 0:
            left()
        forward();
        totalReward+=1

        if(keys[pg.K_o]):
            stop = True;
        #Draw the track
        drawTrack(events, window);
        drawRewardGates(events, window);
        #calculate some car stuff
        rotCar = pg.transform.rotate(car, angle)
        car_rect = rotCar.get_rect(center = car_rect.center)
        endpoint = []  
        #car movement and stuff
        window.blit(rotCar, car_rect)
        for i in range(1, 4):
            degree = angle+i*20
            endX = np.cos(np.radians(degree))*(car_rect.center[0] - car_rect.center[0]) - np.sin(np.radians(degree))*(car_rect.center[1]+200 - car_rect.center[1]) + car_rect.center[0]
            endY = np.sin(np.radians(degree))*(car_rect.center[0] - car_rect.center[0]) - np.cos(np.radians(degree))*(car_rect.center[1]+200 - car_rect.center[1]) + car_rect.center[1]
            endpoint.append((endX, endY))
        for h in range(1, 4):
            degree = angle+h*20+280
            endX = np.cos(np.radians(degree))*(car_rect.center[0] - car_rect.center[0]) - np.sin(np.radians(degree))*(car_rect.center[1]+200 - car_rect.center[1]) + car_rect.center[0]
            endY = np.sin(np.radians(degree))*(car_rect.center[0] - car_rect.center[0]) - np.cos(np.radians(degree))*(car_rect.center[1]+200 - car_rect.center[1]) + car_rect.center[1]
            endpoint.append((endX, endY))
        if(drawModeCount == 1 and not drawMode):
            for point in endpoint:
                pg.draw.aaline(window, (0,0,0), car_rect.center, point, False)
        #check for collisions
        distanceInfo_, reward = collisionCheck(endpoint, window, car_rect.center);
        totalReward += reward
        done = False
        if(len(distanceInfo_)!= 0):
            distanceInfo_ = np.array(distanceInfo_)
            done = (distanceInfo_ <= 20).any()
        if(len(distanceInfo) == 0):
            distanceInfo = distanceInfo_
            #print(distanceInfo)
        #===========================================================================
        #===========================================================================
        #===========================================================================
    #====================================ACCESS RESULTS BASED ON STEP===============
    #===============================================================================
    #===============================================================================
    #===============================================================================
        #current state - distanceInfo
        #action taken - action
        #current reward - totalReward
        #next state - distanceInfo_
        score += totalReward
        if(score > 10000000):
            run = False
        if done:
            totalReward = -100
        distanceInfo = np.reshape(distanceInfo, [1, 6])
        distanceInfo_ = np.reshape(distanceInfo_, [1, 6])
        agent.store(distanceInfo, action, totalReward, distanceInfo_, done)
        distanceInfo = distanceInfo_.copy()
        if(done):
            #print("Finished game: ", ai, " Epsilon is currently at: ", agent.epsilon)
            run = False
        pg.display.update()
    if len(agent.memory)>learnSize:
            agent.learn(learnSize)
    scores.append(score)
    epsilons.append(agent.epsilon)
    #print("Score: ", score)
    pg.quit()
    if(score > 10000000):
        break;